In [1]:
from camCalibration import *
from Modbus_DW import *

cam1=Charuco()
#cam1.readCameraConfig("./pic_highres/data")
# cam1.Calibration(imgPath="./8_3_high",show=False)
# cam1.HandEyeCalibration_HandInEye()#new 按照數字讀圖
# cam1=Charuco()
#cam1.readCameraConfig("./pic_highres/data")
cam1.Calibration(imgPath="./picture/8_3_3_high",show=False)
cam1.HandEyeCalibration_EyeInHand()#new 按照數字讀圖

# c=connectRobot(SERVER_HOST = "192.168.1.1")
# RWTool(c,path="./picture/8_3_3_high")



[[917.31791866   0.         640.58334633]
 [  0.         921.29657002 351.55788766]
 [  0.           0.           1.        ]]
[[ 9.31302546e-02  2.88921840e-01  5.60167537e-04 -5.47268159e-03
  -1.63561171e+00]]
error: 0.3216717748710815
distortion: [[ 9.31302546e-02  2.88921840e-01  5.60167537e-04 -5.47268159e-03
  -1.63561171e+00]]
目錄存在。
write charuco pose finish with  25 imgs
Camera Tool Size:
 Width(mm):  102.90229300123517  Height(mm):  37.05430555438235  Angle(degree):  -17.037969188668907
Camera Tool Orientation:
 Rx:  -0.03684360466233889  Ry:  -0.5139679622380616  Rz:  87.99906944235656


In [2]:
import yaml
path="./picture/8_3_3_high"
readTool(path)
with open(path+"/data/CameraConfig.yaml", 'r') as f:
    data = yaml.load(f, Loader=yaml.Loader)
data

{'ARUCO_size': 4,
 'dist': array([[ 9.31302546e-02,  2.88921840e-01,  5.60167537e-04,
         -5.47268159e-03, -1.63561171e+00]]),
 'iMtx': array([[917.31791866,   0.        , 640.58334633],
        [  0.        , 921.29657002, 351.55788766],
        [  0.        ,   0.        ,   1.        ]]),
 'markerL': 0.015,
 'squareL': 0.02,
 'x': 9,
 'y': 12}

In [5]:
closeRobot(c)

stop DRA!


In [1]:
# System information:
# - Linux Mint 18.1 Cinnamon 64-bit
# - Python 2.7 with OpenCV 3.2.0

import numpy as np
import cv2
from cv2 import aruco
import pickle
import glob
import math

# ChAruco board variables
# CHARUCOBOARD_ROWCOUNT = 12
# CHARUCOBOARD_COLCOUNT = 9 
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_4X4_250)
# ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_6X6_250)


# Create constants to be passed into OpenCV and Aruco methods
# CHARUCO_BOARD = aruco.CharucoBoard_create(
#         squaresX=14,
#         squaresY=10,
#         squareLength=0.01,
#         markerLength=0.007,
#         dictionary=ARUCO_DICT)
# board = aruco.CharucoBoard_create(9, 12, 0.02, 0.015, aruco_dict)
CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=9,
        squaresY=12,
        squareLength=0.02,
        markerLength=0.015,
        dictionary=ARUCO_DICT)




# Create the arrays and variables we'll use to store info like corners and IDs from images processed
corners_all = [] # Corners discovered in all images processed
ids_all = [] # Aruco ids corresponding to corners discovered
image_size = None # Determined at runtime
imgs=[]

# This requires a set of images or a video taken with the camera you want to calibrate
# I'm using a set of images taken with the camera with the naming convention:
# 'camera-pic-of-charucoboard-<NUMBER>.jpg'
# All images used should be the same size, which if taken with the same camera shouldn't be a problem
images = glob.glob('./picture/8_3_high2/*_color.jpg')
# images = glob.glob('./pic3/*_color.jpg')

# Loop through images glob'ed
for iname in images:
    # Open the image
    img = cv2.imread(iname)
    # Grayscale the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find aruco markers in the query image
    corners, ids, _ = aruco.detectMarkers(
            image=gray,
            dictionary=ARUCO_DICT)
        # If a Charuco board was found, let's collect image/corner points
    # Requiring at least 20 squares
    if len(corners) > 10:
    # Outline the aruco markers found in our query image
        img = aruco.drawDetectedMarkers(
                image=img, 
                corners=corners)

        # Get charuco corners and ids from detected aruco markers
        response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
                markerCorners=corners,
                markerIds=ids,
                image=gray,
                board=CHARUCO_BOARD)


        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)
        
        # Draw the Charuco board we've detected to show our calibrator the board was properly detected
        img = aruco.drawDetectedCornersCharuco(
                image=img,
                charucoCorners=charuco_corners,
                charucoIds=charuco_ids)
        
        # If our image size is unknown, set it now
        if not image_size:
            image_size = gray.shape[::-1]
    
        # Reproportion the image, maxing width or height at 1000
        proportion = max(img.shape) / 1000.0
        img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))
        # Pause to display each image, waiting for key press
        cv2.imshow(iname, img)
        cv2.waitKey(0)
    else:
        cv2.imshow('Charuco board', img)
        cv2.waitKey(0)
        print("Not able to detect a charuco board in image: {}".format(iname))

    # Destroy any open CV windows
    cv2.destroyAllWindows()

# Make sure at least one image was found
if len(images) < 1:
    # Calibration failed because there were no images, warn the user
    print("Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file.")
    # Exit for failure
    exit()

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
    # Exit for failure
    exit()

# Now that we've seen all of our images, perform the camera calibration
# based on the set of points we've discovered
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)
    
# Print matrix and distortion coefficient to the console
print(cameraMatrix)
print(distCoeffs)
    
# Save values to be used where matrix+dist is required, for instance for posture estimation
# I save files in a pickle file, but you can use yaml or whatever works for you
f = open('calibration.pckl', 'wb')
pickle.dump((cameraMatrix, distCoeffs, rvecs, tvecs), f)
f.close()
    
# Print to console our success
print('Calibration successful. Calibration file used: {}'.format('calibration.pckl'))
print("error:",calibration)

[[919.5861236    0.         641.38367948]
 [  0.         924.60020836 352.86391377]
 [  0.           0.           1.        ]]
[[ 0.14943493 -0.23997318  0.00054742 -0.00434003 -0.28933569]]
Calibration successful. Calibration file used: calibration.pckl
error: 0.32133489939044263


In [2]:
def AngleAxis2EulerZYX(rvec):
    
    R=cv2.Rodrigues(rvec)
    return Mat2Euler(R[0])

def Mat2Euler(_R):
#     sy = math.sqrt(_R.at<double>(0,0) * _R.at<double>(0,0) +  _R.at<double>(1,0) * _R.at<double>(1,0) );
#     sy = math.sqrt(_R[0,0] * _R[0,0] +  _R[0,1] * _R[0,1] );
    sy = math.sqrt(_R[0,0] * _R[0,0] +  _R[1,0] * _R[1,0] );
    if (sy < 1e-6): # If
        singular=True
    else:
        singular=False
#     float x, y, z;
    if (not singular):
#         x = atan2(_R.at<double>(2,1) , _R.at<double>(2,2));
#         y = atan2(-_R.at<double>(2,0), sy);
#         z = atan2(_R.at<double>(1,0), _R.at<double>(0,0));
        x = math.atan2(_R[2,1] , _R[2,2]);
        y = math.atan2(-_R[2,0], sy);
        z = math.atan2(_R[1,0], _R[0,0]);
    else:
#         x = atan2(-_R.at<double>(1,2), _R.at<double>(1,1));
#         y = atan2(-_R.at<double>(2,0), sy);
#         z = 0;
        x = math.atan2(-_R[1,2], _R[1,1]);
        y = math.atan2(-_R[2,0], sy);
        z = 0;
    
    return [x, y, z]


In [3]:
ret3,rvec3,tvec3=aruco.estimatePoseCharucoBoard(corners_all[0],ids_all[0], CHARUCO_BOARD,cameraMatrix,distCoeffs,np.array([.0,.0,.0]),np.array([.0,.0,.0]))

# ret3,rvec3,tvec3=aruco.estimatePoseCharucoBoard(corners_all[1],ids_all[1], CHARUCO_BOARD,cameraMatrix,distCoeffs,rvecs[10],tvecs[10])
print("t:",tvec3)
print("r:",rvec3)
imgs=images
img0 = cv2.imread(images[0])


Euler = AngleAxis2EulerZYX(rvec3);
# Mat TransO,R;
tx = (CHARUCO_BOARD.getChessboardSize()[0]) * (CHARUCO_BOARD.getSquareLength())/2
ty = (CHARUCO_BOARD.getChessboardSize()[1]) * (CHARUCO_BOARD.getSquareLength())/2
# TransO = (Mat_<double>(3,1) <<tx,ty,0);
TransO = np.array([tx,ty,0.])

R=cv2.Rodrigues(rvec3)
print(R[0])
# Trans_tvec = R * TransO    C++
Trans_tvec = R[0].dot(TransO)
CharucoPose=[0,0,0,0,0,0]
for i in range(3):
    tvec3[i]+=Trans_tvec[i]
#     tvec[i]=tvec[i]+Trans_tvec.at<double>(i,0)
    CharucoPose[i]  = tvec3[i]*1000
    CharucoPose[i+3]= Euler[i]*180/math.pi
print("charucoPose:",CharucoPose)
imgNew=aruco.drawAxis(img0,cameraMatrix,distCoeffs,rvec3,tvec3,0.1)
cv2.imshow('Charuco board Axis', imgNew)
cv2.waitKey(0)

# TransO = (Mat_<double>(3,1) <<tx,ty,0);
TransX = np.array([tx,0.,0.])

R=cv2.Rodrigues(rvec3)
print(R[0])
# Trans_tvec = R * TransO    C++
Trans_tvecX = R[0].dot(TransX)
print("trans",Trans_tvecX)

CharucoPoseX=[0,0,0,0,0,0]
tvec4=np.array([0.,0.,0.])
for i in range(3):
    tvec4[i]=tvec3[i]+Trans_tvecX[i]
#     tvec[i]=tvec[i]+Trans_tvec.at<double>(i,0)
    CharucoPoseX[i]  = tvec4[i]*1000
    CharucoPoseX[i+3]= Euler[i]*180/math.pi

print("charucoPoseX:",CharucoPoseX)
imgNew2=aruco.drawAxis(imgNew,cameraMatrix,distCoeffs,rvec3,tvec4,0.1)

cv2.imshow('Charuco board Axis X', imgNew2)
cv2.waitKey(0)
# print("new T",tvec3)

# TransO = (Mat_<double>(3,1) <<tx,ty,0);
TransY = np.array([0.,ty,0.])

# Trans_tvec = R * TransO    C++
Trans_tvecY = R[0].dot(TransY)
print("trans",Trans_tvecY)

CharucoPoseY=[0,0,0,0,0,0]
tvec5=np.array([0.,0.,0.])
for i in range(3):
    tvec5[i]=tvec3[i]+Trans_tvecY[i]
#     tvec[i]=tvec[i]+Trans_tvec.at<double>(i,0)
    CharucoPoseY[i]  = tvec5[i]*1000
    CharucoPoseY[i+3]= Euler[i]*180/math.pi

print("charucoPoseX:",CharucoPoseY)
imgNew3=aruco.drawAxis(imgNew2,cameraMatrix,distCoeffs,rvec3,tvec5,0.1)

# Reproportion the image, maxing width or height at 1000
proportion = max(imgNew.shape) / 1000.0
imgNew = cv2.resize(imgNew, (int(imgNew.shape[1]/proportion), int(imgNew.shape[0]/proportion)))
cv2.imshow('Charuco board Axis Y', imgNew3)
cv2.waitKey(0)
cv2.destroyAllWindows()

t: [0.10242528 0.10390884 0.38233079]
r: [-2.15850305  2.22798962 -0.08732225]
[[-0.03206309 -0.99726533  0.06658686]
 [-0.99941772  0.03121183 -0.01378572]
 [ 0.01166972 -0.06699011 -0.99768539]]
charucoPose: [-20.132240505307415, 17.706671425894147, 375.34225049869843, -176.15861111556066, -0.6686410578508533, -91.83751982607535]
[[-0.03206309 -0.99726533  0.06658686]
 [-0.99941772  0.03121183 -0.01378572]
 [ 0.01166972 -0.06699011 -0.99768539]]
trans [-0.00288568 -0.08994759  0.00105028]
charucoPoseX: [-23.017918561706765, -72.24092121763623, 376.39252555320843, -176.15861111556066, -0.6686410578508533, -91.83751982607535]
trans [-0.11967184  0.00374542 -0.00803881]
charucoPoseX: [-139.8040779875197, 21.45209052273382, 367.3034380435001, -176.15861111556066, -0.6686410578508533, -91.83751982607535]
